In [17]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt
import pickle
from matplotlib.image import imread


In [18]:
# Load cached data
if not os.path.exists('../data'):
    os.system('gdown 1h7S6N_Rx7gdfO3ZunzErZy6H7620EbZK -O ../data.tar.gz')
    os.system('tar -xf ../data.tar.gz -C ../')
    os.system('rm ../data.tar.gz')
if not os.path.exists('../data/imagenet/human_readable_labels.json'):
    !wget -nv -O ../data/imagenet/human_readable_labels.json -L https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json

data = np.load('../data/imagenet/imagenet-resnet152.npz')
example_paths = os.listdir('../data/imagenet/examples')
smx = data['smx']
labels = data['labels'].astype(int)


In [19]:
# Problem setup
n = 1000 # number of calibration points
alpha = 0.1 # 1-alpha is the desired coverage


In [20]:
# Split the softmax scores into calibration and validation sets (save the shuffling)
idx = np.array([1] * n + [0] * (smx.shape[0]-n)) > 0
np.random.shuffle(idx)
cal_smx, val_smx = smx[idx,:], smx[~idx,:]
cal_labels, val_labels = labels[idx], labels[~idx]


In [21]:
qhat = 1 - alpha
val_pi = val_smx.argsort(1)[:,::-1]
val_srt = np.take_along_axis(val_smx,val_pi,axis=1).cumsum(axis=1)
prediction_sets = np.take_along_axis(val_srt <=
                                      qhat, val_pi.argsort(axis=1), axis=1)

empirical_coverage = prediction_sets[np.arange(prediction_sets.shape[0]),val_labels].mean()
print(f"The empirical coverage is: {empirical_coverage}")

The empirical coverage is: 0.3006938775510204


In [22]:
# 1: get conformal scores. n = calib_Y.shape[0]
cal_scores = 1-cal_smx[np.arange(n),cal_labels]
# 2: get adjusted quantile
q_level = np.ceil((n+1)*(1-alpha))/n
qhat = np.quantile(cal_scores, q_level, interpolation='higher')
prediction_sets = val_smx >= (1-qhat) # 3: form prediction sets

# Calculate empirical coverage
empirical_coverage = prediction_sets[np.arange(prediction_sets.shape[0]),val_labels].mean()
print(f"The empirical coverage is: {empirical_coverage}")

set_sizes = prediction_sets.sum(1)
print(f"The sets sizes are: {np.round(set_sizes.mean(), 1)} ({np.round(set_sizes.std(), 2)}))")

The empirical coverage is: 0.8936938775510204
The sets sizes are: 1.7 (1.05))


In [25]:
cal_pi[0][:5]

array([ 33,  36,   0, 391, 397])

In [28]:
cal_smx[0,33], cal_smx[0,36], cal_smx[0,0]

(0.6325930953025818, 0.06433039158582687, 0.058940235525369644)

In [24]:
cal_pi = cal_smx.argsort(1)[:,::-1]
cal_srt = np.take_along_axis(cal_smx, cal_pi, axis=1).cumsum(axis=1)


In [23]:
# Get scores. calib_X.shape[0] == calib_Y.shape[0] == n
cal_pi = cal_smx.argsort(1)[:,::-1]
cal_srt = np.take_along_axis(cal_smx,cal_pi,axis=1).cumsum(axis=1)
cal_scores = np.take_along_axis(cal_srt,cal_pi.argsort(axis=1),axis=1)[range(n),cal_labels]

# Get the score quantile
qhat = np.quantile(cal_scores, np.ceil((n+1)*(1-alpha))/n, interpolation='higher')

# Deploy (output=list of length n, each element is tensor of classes)
val_pi = val_smx.argsort(1)[:,::-1]
val_srt = np.take_along_axis(val_smx,val_pi,axis=1).cumsum(axis=1)
prediction_sets = np.take_along_axis(val_srt <= qhat,val_pi.argsort(axis=1),axis=1)

empirical_coverage = prediction_sets[np.arange(prediction_sets.shape[0]),val_labels].mean()
print(f"The empirical coverage is: {empirical_coverage}")

set_sizes = prediction_sets.sum(1)
print(f"The sets sizes are: {np.round(set_sizes.mean(), 1)} ({np.round(set_sizes.std(), 2)}))")

The empirical coverage is: 0.9147755102040817
The sets sizes are: 243.4 (302.69))


In [ ]:
# imagenet
# https://github.com/pytorch/examples/tree/main/imagenet

In [ ]:
# CIFAR100

Using cache found in /home/royhirsch/.cache/torch/hub/chenyaofo_pytorch-cifar-models_master


In [56]:
import torch
import torch.nn as nn

class EmbedResnet(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def __call__(self, x):
        x = self.model.conv1(x)
        x = self.model.bn1(x)
        x = self.model.relu(x)

        x = self.model.layer1(x)
        x = self.model.layer2(x)
        x = self.model.layer3(x)

        x = self.model.avgpool(x)
        return x.view(x.size(0), -1)

    def fc(self, x):
        return self.model.fc(x)

class EmbedRepVGG(nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def __call__(self, x):
        out = self.model.stage0(x)
        out = self.model.stage1(out)
        out = self.model.stage2(out)
        out = self.model.stage3(out)
        out = self.model.stage4(out)
        out = self.model.gap(out)
        out = out.view(out.size(0), -1)
        return out

    def fc(self, x):
        return self.model.linear(x)

# https://github.com/chenyaofo/pytorch-cifar-models
model = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar100_repvgg_a2", pretrained=True)
embed_model = EmbedRepVGG(model)
embed_model(torch.rand(1,3,224,224)).shape

Using cache found in /home/royhirsch/.cache/torch/hub/chenyaofo_pytorch-cifar-models_master


torch.Size([1, 1408])

In [57]:
import numpy as np
import torchvision
import torchvision.transforms as tt

_MEAN = [0.5070751592371323, 0.48654887331495095, 0.4409178433670343]
_STD = [0.26733428587941854, 0.25643846292120615, 0.2761504713263903]


transform_train = tt.Compose([tt.RandomCrop(32, padding=4,padding_mode='reflect'), 
                              tt.RandomHorizontalFlip(), 
                              tt.ToTensor(), 
                              tt.Normalize(_MEAN,_STD,inplace=True)])
transform_test = tt.Compose([tt.ToTensor(), tt.Normalize(_MEAN,_STD)])

batch_size = 32
num_workers = 1
trainset = torchvision.datasets.CIFAR100("./",
                                         train=True,
                                         download=True,
                                         transform=transform_test)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size, shuffle=True, num_workers=num_workers,pin_memory=True)

testset = torchvision.datasets.CIFAR100("./",
                                        train=False,
                                        download=True,
                                        transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size,pin_memory=True, num_workers=num_workers)


Files already downloaded and verified
Files already downloaded and verified


In [54]:
def get_embeds(device, loader):
    model = embed_model.to(device)
    model.eval()

    all_embeds = []
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch in loader:
            x = batch[0].to(device)
            embeds = embed_model(x)
            preds = embed_model.fc(embeds)

            embeds = embeds.detach().cpu().numpy()
            preds = preds.detach().cpu().numpy()

            all_embeds.append(embeds)
            all_preds.append(preds)
            all_labels.append(batch[1].numpy())

    all_embeds = np.concatenate(all_embeds, 0)
    all_preds = np.concatenate(all_preds, 0)
    all_labels = np.concatenate(all_labels, 0)
    return all_embeds, all_preds, all_labels

def save_pickle(data, file_path):
    with open(file_path, 'wb') as file:
        pickle.dump(data, file)

In [58]:
device = torch.device('cuda:6')
out_dir = '/home/royhirsch/conformal/cifar100_repvgg_a2'
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

train_embeds, train_preds, train_labels = get_embeds(device, trainloader)
print(train_embeds.shape)
print((train_preds.argmax(1) == train_labels).mean())
save_pickle({'embeds': train_embeds,
             'preds': train_preds,
             'labels': train_labels}, os.path.join(out_dir, 'train.pickle'))
test_embeds, test_preds, test_labels = get_embeds(device, testloader)
print(test_embeds.shape)
print((test_preds.argmax(1) == test_labels).mean())
save_pickle({'embeds': test_embeds,
             'preds': test_preds,
             'labels': test_labels}, os.path.join(out_dir, 'test.pickle'))


(50000, 1408)
0.99808
(10000, 1408)
0.748


In [64]:
import timm 
timm.list_models('*100*')

['fbnetc_100',
 'ghostnet_100',
 'lcnet_100',
 'mnasnet_100',
 'mobilenetv2_100',
 'mobilenetv3_large_100',
 'mobilenetv3_small_100',
 'mobilevitv2_100',
 'rexnet_100',
 'rexnetr_100',
 'semnasnet_100',
 'spnasnet_100',
 'tf_mobilenetv3_large_100',
 'tf_mobilenetv3_large_minimal_100',
 'tf_mobilenetv3_small_100',
 'tf_mobilenetv3_small_minimal_100']

In [108]:
!kaggle competitions download -c imagenet-object-localization-challenge



5475.42s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
403 - Forbidden - You must accept this competition's rules before you'll be able to download files.
